# Transfer Learning in Keras

In this notebook, we'll cover how to load a pre-trained model (in this case, VGGNet19) and finetune it for a new task: detecting hot dogs.

#### Load dependencies

In [ ]:
from keras.applications.vgg19 import VGG19
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import TensorBoard

#### Load the pre-trained VGG19 model

In [ ]:
vgg19 = VGG19(include_top=False,
              weights='imagenet',
              input_shape=(224,224,3),
              pooling=None)

#### Freeze all the layers in the base VGGNet19 model

In [ ]:
for layer in vgg19.layers:
    layer.trainable = False

#### Add custom classification layers

In [ ]:
# Instantiate the sequential model and add the VGG19 model:
model = Sequential()
model.add(vgg19)

# Add the custom layers atop the VGG19 model:
model.add(Flatten(name='flattened'))
model.add(Dropout(0.5, name='dropout'))
model.add(Dense(2, activation='softmax', name='predictions'))

#### Compile the model for training

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#### Prepare the data for training

The dataset is available for download [here](https://www.kaggle.com/dansbecker/hot-dog-not-hot-dog/home).  
Instead of downloading and copying the data to Google Drive, we can use API to download any data from Kaggle.

In [ ]:
# [1] sign in/register to Kaggle account
# [2] from account settings, API "create new token" and download kaggle.json
# [3] upload kaggle.json to colab sample_data

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/sample_data/'

!chmod 600 /content/sample_data/kaggle.json     # optional security measure

In [ ]:
!kaggle datasets download -d dansbecker/hot-dog-not-hot-dog

In [ ]:
!unzip \*.zip

In [ ]:
# Instantiate two image generator classes:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last',
    rotation_range=30,
    horizontal_flip=True,
    fill_mode='reflect')

valid_datagen = ImageDataGenerator(
    rescale=1.0/255,
    data_format='channels_last')

In [ ]:
# Define the batch size:
batch_size=32

In [ ]:
# Define the train and validation generators:
train_generator = train_datagen.flow_from_directory(
    directory='./train',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

valid_generator = valid_datagen.flow_from_directory(
    directory='./test',
    target_size=(224, 224),
    classes=['hot_dog','not_hot_dog'],
    class_mode='categorical',
    batch_size=batch_size,
    shuffle=True,
    seed=42)

In [ ]:
tensorboard = TensorBoard('logs/transfer')

In [ ]:
model.fit(train_generator,
          steps_per_epoch=15,
          epochs=20,
          validation_data=valid_generator,
          validation_steps=15,
          callbacks=[tensorboard])

In [ ]:
%reload_ext tensorboard
%tensorboard --logdir logs/transfer